In [1]:
from QF_FB_C.lib_mlp import *
from QF_FB_C.lib_qf_fb import *
from QF_Net.lib_qf_net import *
from QF_Net.lib_util import *

import argparse
import time
import torch
import torchvision.transforms as transforms
from torchvision import datasets
import torch.nn as nn
import os
import sys

from collections import Counter
from pathlib import Path

In [9]:

def train(model, optimizer, device, epoch,interest_num,criterion,train_loader):
    model.train()
    correct = 0
    epoch_loss = []
    for batch_idx, (data, target) in enumerate(train_loader):
        target, new_target = modify_target(target,interest_num)

        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data, True)

        pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        loss = criterion(output, target)
        epoch_loss.append(loss.item())
        loss.backward()

        optimizer.step()

        if batch_idx % 50 == 0:            
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {}/{} ({:.2f}%)'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss, correct, (batch_idx + 1) * len(data),
                       100. * float(correct) / float(((batch_idx + 1) * len(data)))))
    print("-" * 20, "training done, loss", "-" * 20)    

def test(model,device,interest_num,criterion,test_loader):
    accur = []
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        target, new_target = modify_target(target,interest_num)
        data, target = data.to(device), target.to(device)        
        output = model(data, False)        
        test_loss += criterion(output, target)  # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    a = 100. * correct / len(test_loader.dataset)
    accur.append(a)
    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * float(correct) / float(len(test_loader.dataset))))

    return float(correct) / len(test_loader.dataset)


def load_data(interest_num,img_size,batch_size,inference_batch_size,num_workers):
    # convert data to torch.FloatTensor
    transform = transforms.Compose([transforms.Resize((img_size, img_size)), transforms.ToTensor()])    
    train_data = datasets.MNIST(root='data', train=True,
                                download=True, transform=transform)
    test_data = datasets.MNIST(root='data', train=False,
                               download=True, transform=transform)
    train_data = select_num(train_data, interest_num)
    test_data = select_num(test_data, interest_num)

    # prepare data loaders
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                               num_workers=num_workers, shuffle=True, drop_last=True)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=inference_batch_size,
                                              num_workers=num_workers, shuffle=False, drop_last=True)

    return train_loader,test_loader

In [10]:
print("=" * 100)
print("Training procedure for Quantum Computer:")
print("\tStart at:", time.strftime("%m/%d/%Y %H:%M:%S"))
print("\tProblems and issues, please contact Dr. Weiwen Jiang (wjiang2@nd.edu)")
print("\tEnjoy and Good Luck!")
print("=" * 100)
print()


device = "cpu"
interest_class = [3,6]
img_size = 4
num_workers = 0
batch_size = 32
inference_batch_size = 32
layers = [4,2]
init_lr = 0.1
milestones = [1]
max_epoch = 2
resume_path = ""
training = True
binary = False
debug = False
classic = False
init_qc_lr = 0.1
with_norm = True
sim_range = []
given_ang = [[1,-1,1,-1],[-1,-1]]
train_ang = True
save_chkp = False
epoch_init, acc = 0, 0


# Load data and create model
train_loader, test_loader = load_data(interest_class,img_size,batch_size,inference_batch_size,num_workers)
criterion = nn.CrossEntropyLoss()
model = Net(img_size,layers,with_norm,given_ang,train_ang,training,binary,classic).to(device)

# Initialize Normalization Parameters
para_list = []
for idx in range(len(layers)):
    fc = getattr(model, "fc"+str(idx))
    IAdj = getattr(model, "IAdj"+str(idx))
    para_list.append({'params': fc.parameters(), 'lr': init_lr})
    para_list.append({'params': IAdj.parameters(), 'lr': init_qc_lr})
optimizer = torch.optim.Adam(para_list)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones, gamma=0.1)

# Training

for epoch in range(epoch_init, max_epoch + 1):
    print("=" * 20, epoch, "epoch", "=" * 20)
    print("Epoch Start at:", time.strftime("%m/%d/%Y %H:%M:%S"))
    print("-" * 20, "learning rates", "-" * 20)
    for param_group in optimizer.param_groups:
        print(param_group['lr'], end=",")
    print()
    print("-" * 20, "training", "-" * 20)
    print("Trainign Start at:", time.strftime("%m/%d/%Y %H:%M:%S"))
    train(model, optimizer, device, epoch,interest_class,criterion,train_loader)
    print("Trainign End at:", time.strftime("%m/%d/%Y %H:%M:%S"))
    print("-" * 60)
    print()
    print("-" * 20, "testing", "-" * 20)
    print("Testing Start at:", time.strftime("%m/%d/%Y %H:%M:%S"))
    cur_acc = test(model,device,interest_class,criterion,test_loader)
    print("Testing End at:", time.strftime("%m/%d/%Y %H:%M:%S"))
    print("-" * 60)
    print()

    scheduler.step()

    is_best = False
    if cur_acc > acc:
        is_best = True
        acc = cur_acc
    print("Best accuracy: {}; Current accuracy {}. Checkpointing".format(acc, cur_acc))
    print("Epoch End at:", time.strftime("%m/%d/%Y %H:%M:%S"))
    print("=" * 60)
    print()

Training procedure for Quantum Computer:
	Start at: 07/02/2020 08:49:57
	Problems and issues, please contact Dr. Weiwen Jiang (wjiang2@nd.edu)
	Enjoy and Good Luck!

==================== 0 epoch ====================
Epoch Start at: 07/02/2020 08:49:58
-------------------- learning rates --------------------
0.1,0.1,0.1,0.1,
-------------------- training --------------------
Trainign Start at: 07/02/2020 08:49:58
Train Epoch: 0 [0/12049 (0%)]	Loss: 0.692409	Accuracy: 22/32 (68.75%)
Train Epoch: 0 [1600/12049 (13%)]	Loss: 0.671228	Accuracy: 1530/1632 (93.75%)
Train Epoch: 0 [3200/12049 (27%)]	Loss: 0.671653	Accuracy: 3077/3232 (95.20%)
Train Epoch: 0 [4800/12049 (40%)]	Loss: 0.673316	Accuracy: 4616/4832 (95.53%)
Train Epoch: 0 [6400/12049 (53%)]	Loss: 0.669353	Accuracy: 6141/6432 (95.48%)
Train Epoch: 0 [8000/12049 (66%)]	Loss: 0.669420	Accuracy: 7670/8032 (95.49%)
Train Epoch: 0 [9600/12049 (80%)]	Loss: 0.677605	Accuracy: 9199/9632 (95.50%)
Train Epoch: 0 [11200/12049 (93%)]	Loss: 0.666